#Carga de datos
Partimos de la exisitencia de un archivo .csv extraido de esta [pagina](https://www.kaggle.com/amritvirsinghx/web-series-ultimate-edition).

Cargamos el csv en una variable pandas


In [ ]:
!wget https://github.com/ElBley/ABP/raw/main/All_Streaming_Shows.csv

--2020-11-29 02:04:29--  https://github.com/ElBley/ABP/raw/main/All_Streaming_Shows.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ElBley/ABP/main/All_Streaming_Shows.csv [following]
--2020-11-29 02:04:30--  https://raw.githubusercontent.com/ElBley/ABP/main/All_Streaming_Shows.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9095510 (8.7M) [text/plain]
Saving to: ‘All_Streaming_Shows.csv’

All_Streaming_Shows 100%[===================>]   8.67M  --.-KB/s    in 0.08s   

2020-11-29 02:04:30 (109 MB/s) - ‘All_Streaming_Shows.csv’ saved [9095510/9095510]



In [ ]:
import pandas
data = pandas.read_csv('All_Streaming_Shows.csv')

Como vamos a utilizar la descripción para aplicar el algoritmo de recomendación, las series sin descripción no nos son útiles, por lo que las borramos, y reseteamos el indice para poder manipular las filas más adelante.

In [ ]:
data = data[data['Description'] != '-1']
data = data.reset_index()
data

,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform
0,0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix
1,1,Game of Thrones,2011,18+,9.3,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO"
2,2,Rick and Morty,2013,18+,9.2,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu"
3,3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix
4,4,The Boys,2019,18+,8.7,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video
...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,NaN,NaN,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,Hulu
11870,12348,A Fishing Story with Ronnie Green,2017,NaN,NaN,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,"Prime Video,fuboTV"
11871,12350,NHL Road to the Outdoor Classics,2016,NaN,NaN,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,"Prime Video,Epix"
11872,12351,Addy Media,2018,NaN,NaN,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,Prime Video


Vemos que para el tipo de dato 'Content Rating' existen varios posibles valores, y el valor "nan", como esto es un dato nulo, vamos a asumir que todas las series con valor "nan" son para todos los publicos, y vamos a subsituirlo por el valor "all"

In [ ]:
data['Content Rating'].unique()

array(['18+', '16+', '7+', 'all', nan, '13+'], dtype=object)

In [ ]:
data['Content Rating'] = data['Content Rating'].fillna('all')
data

,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform
0,0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix
1,1,Game of Thrones,2011,18+,9.3,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO"
2,2,Rick and Morty,2013,18+,9.2,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu"
3,3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix
4,4,The Boys,2019,18+,8.7,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video
...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,NaN,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,Hulu
11870,12348,A Fishing Story with Ronnie Green,2017,all,NaN,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,"Prime Video,fuboTV"
11871,12350,NHL Road to the Outdoor Classics,2016,all,NaN,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,"Prime Video,Epix"
11872,12351,Addy Media,2018,all,NaN,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,Prime Video


El valor de no of seasons es una cadena de texto, nos interesa más convertirla en un valor entero, por lo que iteramos por todo el dataset, convirtiendo cada valor en un entero en una nueva columna. (Para esta operacion necesitamos que los índices estén consecutivos).

In [ ]:
import re
p = re.compile('-?[0-9]+')
var = 0
data['Number of Seasons'] = 0
for row in data.itertuples():
  data['Number of Seasons'][var] = int(p.match(row[9]).group())
  var +=1
print(var)
data


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


11874


,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform,Number of Seasons
0,0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix,5
1,1,Game of Thrones,2011,18+,9.3,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO",8
2,2,Rick and Morty,2013,18+,9.2,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu",4
3,3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix,3
4,4,The Boys,2019,18+,8.7,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video,2
...,...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,NaN,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,Hulu,1
11870,12348,A Fishing Story with Ronnie Green,2017,all,NaN,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,"Prime Video,fuboTV",2
11871,12350,NHL Road to the Outdoor Classics,2016,all,NaN,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,"Prime Video,Epix",1
11872,12351,Addy Media,2018,all,NaN,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,Prime Video,1


Los datos de puntuación de imdb y R vamos a realizar una media con ellos. Como existen datos NaN para el IMDB tenemos que normalizarlos, en este caso lo ponemos a -1 ya que es el valor que tiene R rating para sus vacios. Como las columnas de imdb y r rating tienen distinta escala, tenemos que poner ambos en la misma escala, en este caso sobre 100 puntos. Una vez tenemos las columnas en el mismo formato, podemos generar una nueva columna con la media de ambas.

In [ ]:
data['IMDB Rating'] = data['IMDB Rating'] * 10
data['IMDB Rating'] = data['IMDB Rating'].fillna(-1)
data['Mean Rating'] = data[['IMDB Rating', 'R Rating']].mean(axis=1)
data

,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform,Number of Seasons,Mean Rating
0,0,Breaking Bad,2008,18+,95.0,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix,5,97.5
1,1,Game of Thrones,2011,18+,93.0,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO",8,96.0
2,2,Rick and Morty,2013,18+,92.0,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu",4,94.5
3,3,Stranger Things,2016,16+,88.0,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix,3,92.0
4,4,The Boys,2019,18+,87.0,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video,2,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,-1.0,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,Hulu,1,-1.0
11870,12348,A Fishing Story with Ronnie Green,2017,all,-1.0,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,"Prime Video,fuboTV",2,-1.0
11871,12350,NHL Road to the Outdoor Classics,2016,all,-1.0,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,"Prime Video,Epix",1,-1.0
11872,12351,Addy Media,2018,all,-1.0,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,Prime Video,1,-1.0


## El formato de la columna Streaming Platform está muy especificado por eso simplemente vamos a dejar como plataformas posibles:
    -Amazon Prime Video
    -Netflix
    -HBO
    -y Others, donde se ubicarán el resto


Primero creamos las columnas correspondientes a estos atributos dummies:




In [ ]:
data.insert(9,'Others',0)
data.insert(9,'Prime Video',0)
data.insert(9,'HBO',0)
data.insert(9,'Netflix',0)

data

,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Netflix,HBO,Prime Video,Others,Streaming Platform,Number of Seasons,Mean Rating
0,0,Breaking Bad,2008,18+,95.0,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,0,0,0,0,Netflix,5,97.5
1,1,Game of Thrones,2011,18+,93.0,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,0,0,0,0,"HBO MAX,HBO",8,96.0
2,2,Rick and Morty,2013,18+,92.0,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,0,0,0,0,"Free Services,HBO MAX,Hulu",4,94.5
3,3,Stranger Things,2016,16+,88.0,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,0,0,0,0,Netflix,3,92.0
4,4,The Boys,2019,18+,87.0,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,0,0,0,0,Prime Video,2,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,-1.0,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,0,0,0,0,Hulu,1,-1.0
11870,12348,A Fishing Story with Ronnie Green,2017,all,-1.0,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,0,0,0,0,"Prime Video,fuboTV",2,-1.0
11871,12350,NHL Road to the Outdoor Classics,2016,all,-1.0,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,0,0,0,0,"Prime Video,Epix",1,-1.0
11872,12351,Addy Media,2018,all,-1.0,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,0,0,0,0,Prime Video,1,-1.0


## Transformamos los "-1" que encontremos en "Not Found" para que podamos trabajar con Strings solamente.

In [ ]:
data['Streaming Platform'] = data['Streaming Platform'].fillna("Not Found")
data1 = data[data["Streaming Platform"] == "-1"]
data1["Streaming Platform"] = "Not Found" 
data[data["Streaming Platform"] == "-1"] = data1
data

,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Netflix,HBO,Prime Video,Others,Streaming Platform,Number of Seasons,Mean Rating
0,0,Breaking Bad,2008,18+,95.0,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,0,0,0,0,Netflix,5,97.5
1,1,Game of Thrones,2011,18+,93.0,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,0,0,0,0,"HBO MAX,HBO",8,96.0
2,2,Rick and Morty,2013,18+,92.0,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,0,0,0,0,"Free Services,HBO MAX,Hulu",4,94.5
3,3,Stranger Things,2016,16+,88.0,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,0,0,0,0,Netflix,3,92.0
4,4,The Boys,2019,18+,87.0,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,0,0,0,0,Prime Video,2,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,-1.0,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,0,0,0,0,Hulu,1,-1.0
11870,12348,A Fishing Story with Ronnie Green,2017,all,-1.0,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,0,0,0,0,"Prime Video,fuboTV",2,-1.0
11871,12350,NHL Road to the Outdoor Classics,2016,all,-1.0,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,0,0,0,0,"Prime Video,Epix",1,-1.0
11872,12351,Addy Media,2018,all,-1.0,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,0,0,0,0,Prime Video,1,-1.0


## Por último recoreremos la columna que contiene la información de las plataformas que ofrecen cada serie y si encaja con alguna de nuestras plataformas elegidas le pondremos un 1 en su variable dummie correspondiente.

In [ ]:
aux = data["Streaming Platform"]
for i in range(aux.size):
      if 'Netflix' in aux[i]:
          data['Netflix'][i] = 1
      if 'Prime Video' in aux[i]:
          data['Prime Video'][i] = 1
      if 'HBO' in aux[i]:
          data['HBO'][i] = 1
      if(not('Netflix' in aux[i])
         and not('Prime Video' in aux[i])
         and not('HBO' in aux[i])):
          data['Others'][i] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: Sett

##Y ya tenemos los datos listos para alimentar el algoritmo

In [ ]:
data

,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Netflix,HBO,Prime Video,Others,Streaming Platform,Number of Seasons,Mean Rating
0,0,Breaking Bad,2008,18+,95.0,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,1,0,0,0,Netflix,5,97.5
1,1,Game of Thrones,2011,18+,93.0,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,0,1,0,0,"HBO MAX,HBO",8,96.0
2,2,Rick and Morty,2013,18+,92.0,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,0,1,0,0,"Free Services,HBO MAX,Hulu",4,94.5
3,3,Stranger Things,2016,16+,88.0,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,1,0,0,0,Netflix,3,92.0
4,4,The Boys,2019,18+,87.0,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,0,0,1,0,Prime Video,2,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,-1.0,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,0,0,0,1,Hulu,1,-1.0
11870,12348,A Fishing Story with Ronnie Green,2017,all,-1.0,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,0,0,1,0,"Prime Video,fuboTV",2,-1.0
11871,12350,NHL Road to the Outdoor Classics,2016,all,-1.0,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,0,0,1,0,"Prime Video,Epix",1,-1.0
11872,12351,Addy Media,2018,all,-1.0,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,0,0,1,0,Prime Video,1,-1.0


In [ ]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

preprocessedText = []

for row in data.itertuples():
    
    
    text = word_tokenize(row[8])
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedData = data
preprocessedData['processed_text'] = preprocessedText

preprocessedData
    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Netflix,HBO,Prime Video,Others,Streaming Platform,Number of Seasons,Mean Rating,processed_text
0,0,Breaking Bad,2008,18+,95.0,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,1,0,0,0,Netflix,5,97.5,when walter white new mexico chemistri teacher...
1,1,Game of Thrones,2011,18+,93.0,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,0,1,0,0,"HBO MAX,HBO",8,96.0,seven nobl famili fight control mythic land we...
2,2,Rick and Morty,2013,18+,92.0,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,0,1,0,0,"Free Services,HBO MAX,Hulu",4,94.5,rick old man recent reconnect famili He spend ...
3,3,Stranger Things,2016,16+,88.0,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,1,0,0,0,Netflix,3,92.0,when young boy vanish small town uncov mysteri...
4,4,The Boys,2019,18+,87.0,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,0,0,1,0,Prime Video,2,91.0,A group vigilant known inform the boy set take...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,-1.0,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,0,0,0,1,Hulu,1,-1.0,mind made easi fun kid discov superpow curios ...
11870,12348,A Fishing Story with Ronnie Green,2017,all,-1.0,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,0,0,1,0,"Prime Video,fuboTV",2,-1.0,A fish stori ronni green one episod stream sub...
11871,12350,NHL Road to the Outdoor Classics,2016,all,-1.0,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,0,0,1,0,"Prime Video,Epix",1,-1.0,road nhl outdoor classic take us deep insid fo...
11872,12351,Addy Media,2018,all,-1.0,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,0,0,1,0,Prime Video,1,-1.0,addi media one episod stream subscript prime v...


In [ ]:
preprocessedData.iloc[0]['Description']

"When Walter White, a New Mexico chemistry teacher, is diagnosed with Stage III cancer and given a prognosis of only two years left to live. He becomes filled with a sense of fearlessness and an unrelenting desire to secure his family's financial future at any cost as he enters the dangerous world of drugs and crime.Breaking Bad featuring Bryan Cranston and Aaron Paul has one or more episodes streaming with subscription on Netflix, available for purchase on iTunes, available for purchase on Google Play, and 3 others. It's a crime and drama show with 62 episodes over 5 seasons. Breaking Bad is no longer running and has no plans to air new episodes or seasons. It has a very high IMDb audience rating of 9.5 (1,391,409 votes) and was very well received by critics."

In [ ]:
preprocessedData.iloc[0]['processed_text']

'when walter white new mexico chemistri teacher diagnos stage iii cancer given prognosi two year left live He becom fill sens fearless unrel desir secur famili financi futur cost enter danger world drug bad featur bryan cranston aaron paul one episod stream subscript netflix avail purchas itun avail purchas googl play 3 other It crime drama show 62 episod 5 season break bad longer run plan air new episod season It high imdb audienc rate vote well receiv critic'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_text'])
textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])

distance_matrix= pairwise_distances(textsBoW,textsBoW ,metric='cosine')

In [ ]:
textsBoW.shape

(11874, 33058)

In [ ]:
#bagOfWordsModel.get_feature_names()

In [ ]:
bagOfWordsModel.get_feature_names()[2210]

'anchor'

In [ ]:
def buscar(serieText):
    searchTitle = serieText 
    indexOfTitle = preprocessedData[preprocessedData['Series Title']==searchTitle].index.values[0]
    distance_scores = list(enumerate(distance_matrix[indexOfTitle]))
    ordered_scores = sorted(distance_scores, key=lambda x: x[1])
    top_scores = ordered_scores[1:11]
    top_indexes = [i[0] for i in top_scores]
    return preprocessedData['Series Title'].iloc[top_indexes]

In [ ]:
searchTitle = "The Boys" #Película base para las recomendaciones
indexOfTitle = preprocessedData[preprocessedData['Series Title']==searchTitle].index.values[0]
indexOfTitle

4

In [ ]:
distance_scores = list(enumerate(distance_matrix[indexOfTitle]))
ordered_scores = sorted(distance_scores, key=lambda x: x[1])
top_scores = ordered_scores[1:11]
top_indexes = [i[0] for i in top_scores]

In [ ]:
distance_matrix

array([[0.        , 0.93902789, 0.9482054 , ..., 0.98950162, 0.97759465,
        0.96954089],
       [0.93902789, 0.        , 0.94058848, ..., 0.98890607, 0.97746615,
        0.97528826],
       [0.9482054 , 0.94058848, 0.        , ..., 0.98301855, 0.97329228,
        0.95696585],
       ...,
       [0.98950162, 0.98890607, 0.98301855, ..., 0.        , 0.95907629,
        0.95340978],
       [0.97759465, 0.97746615, 0.97329228, ..., 0.95907629, 0.        ,
        0.91723382],
       [0.96954089, 0.97528826, 0.95696585, ..., 0.95340978, 0.91723382,
        0.        ]])

In [ ]:
!pip install flask
!pip install flask-ngrok

from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import request

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/", methods = ["GET","POST"])
def home():
  temp = """
    <h1><strong>Recomendador de series Case Sensitive</strong></h1>
    <div>
        <h2>Selecciona una serie:</h2>
        <form action="/" method="POST">
            <input type="text" name="serie">
            <button type="submit" style=" display:block; width: 100px; height: 20.98px;">Buscar</button>
        </form>
    </div>"""
  series = ""
  if request.method == 'POST':
    series = "<ul>"
    serie = request.form['serie']
    try:
      x = buscar(serie)
      for row in x:
        series = series + "<li>" + row + "</li>"
      series = series + "</ul>"
    except:
      series = "No se ha encontrado serie"
    
  
  ret = "<html><body>" + temp + series + "</body></html>"
  return ret
  
app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://47718b960379.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [29/Nov/2020 02:05:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:05:41] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Nov/2020 02:05:47] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:05:51] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:05:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:06:34] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:06:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:07:08] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:10:32] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:10:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:11:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:11:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:12:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:12:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:14:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2020 02:14:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2